In [1]:
!pip install gap-stat kneed

In [2]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import Birch

from tqdm import tqdm
import pandas as pd
import numpy as np

# Importar base de datos

In [ ]:
import requests
import pandas as pd

access_token = ''
file_id = '16f1oPpO_vQSLNEmnQJMOa7lee5c49Uo9'
url = f'https://www.googleapis.com/drive/v3/files/{file_id}?alt=media'

headers = {'Authorization': f'Bearer {access_token}'}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    with open('temp_file.csv', 'wb') as f:
        f.write(response.content)        
    df = pd.read_csv('temp_file.csv')
else:
    print(f'Error al descargar el archivo: {response.status_code}')


# Limpiar base de datos

In [4]:
df = data[:len(data)//100]

In [5]:
import pandas as pd
df = df.fillna(0)
df.loc[df['sex'] == 'F', 'sex'] = 2
df.loc[df['sex'] == 'M', 'sex'] = 1
df.loc[df['sex'] == 'U', 'sex'] = 3
df = df.drop(columns = ['p_id', 'v_o_id', 'stratification'], axis= 1)
#df = np.array(df)

# Calcular k's

## Importar metricas de _k_ values

In [6]:
from k_values.silhouette import silhouette
from k_values.elbow import elbow
from k_values.gap_statistic import gap_statistic

## Evaluar métricas de los _k_ values

In [7]:
Models= [KMeans,Birch,AgglomerativeClustering]
# Models= [KMeans]

Ks=[]

for m in Models:
    Ks.append(silhouette(m,df))

# Ks.append(gap_statistic(np.array(df)))
Ks.append(elbow(KMeans,df))

Ks = list(set(Ks))

print(Ks)

[2, 4]


In [8]:
from metrics.metrics import informe

Models= [KMeans,Birch,AgglomerativeClustering]
for i in Models:
    informe(df,Ks,i)
    

# Evaluando los modelos

In [10]:

def k_means(df, lista_k):
  #df = np.array(db)
  modelos = []
  for k in lista_k:
    # Enternamiento
    km = KMeans(n_clusters=k, random_state=37)
    km.fit(df)
    modelos.append(km)
  return modelos

def metrics(df, l_modelos):
  scores = {'davie_bouldin' : [], 'calinski_harabasz': [], 'sillhouette': []}
  for m in  l_modelos:
    scores['davie_bouldin'].append(davies_bouldin_score(df, m.labels_))
    scores['calinski_harabasz'].append(calinski_harabasz_score(df, m.labels_))
    scores['sillhouette'].append(silhouette_score(df, m.labels_))
  return scores

def informe_metrics(scores, lista_k):
    df = pd.DataFrame(scores)
    df.insert(0, 'numero de k', lista_k)
    #df['davie_nom'] = df['davie_bouldin'].apply(lambda a: 1 / (a + 1))
    #df['calinski_nom'] = df['calinski_harabasz'].apply(lambda a: 1 - (1/ (a + 1)))
    #df['sillhoutte_nom'] = df['sillhouette'].apply(lambda a: (a + 1)/ 2)

    # Calcular la media de las últimas tres métricas
    #df['media metrics'] = df.iloc[:, -3:].sum(axis=1) / 3
    df.to_csv("scores.xlsx", index=False)
    return df


def grafics():
  pass

def main(db):
  df = np.array(db)
  k = [i for i in range(2, 41)]
  modelos = k_means(df, k)
  scores = metrics(df, modelos)
  informe_metrics(scores, k)

  return (modelos, scores)


# 